In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

In [70]:
data = pd.read_csv(r'C:\Users\marte\OneDrive\Documents\Projects!\NBA Model\cleanData.csv')
data

,Date,GameID,Season,HomeAway,HomeTeam,HomeWin,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXHomeAway,XXAwayTeam,AwayWin,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBallTurnovers,XXLiveBallTurnoverPct,XXLostBa

In [71]:
def newElo(List, theDict):
    homePoints = List[8]
    homeElo = theDict.get(List[4])[-1]
    awayPoints = List[131]
    awayElo = theDict.get(List[127])[-1]
    transformHome = 10**(homeElo/400)
    transformAway = 10**(awayElo/400)
    expectHome = transformHome/(transformHome+transformAway)
    expectAway = transformAway/(transformHome+transformAway)
    eloDif = (homeElo+100)-awayElo
    
    if homePoints - awayPoints > 0:
        movm = ((homePoints-awayPoints+3)**0.8)/(7.5+(0.006*eloDif))
        eloChange = int((20*(1-expectHome))*movm)
        newHome = homeElo + eloChange
        newAway = awayElo - eloChange
    else:
        movm = ((awayPoints-homePoints+3)**0.8)/(7.5+(0.006*eloDif))
        eloChange = int((20*(1-expectAway))*movm)
        newAway = awayElo + eloChange
        newHome = homeElo - eloChange
    
    return [List[4],newHome], [List[127],newAway]

In [72]:
def addElos(df):
    
    listOfTeams = ['ATL','BKN','BOS','CHA','CHI','CLE','DAL','DEN','DET','GSW','HOU','IND',
               'LAC','LAL','MEM','MIA','MIL','MIN','NOP','NYK','OKC','ORL','PHI','PHX',
               'POR','SAC','SAS','TOR','UTA','WAS']
    
    oi = []
    for i in range(0,len(data)):
        oi.append(0)
    df.insert(loc=6,column='Elo',value=oi)
    df.insert(loc=129,column='XXElo',value=oi)
    
    listOfSeasons = []
    for i in list(df['Season']):
        if i not in listOfSeasons:
            listOfSeasons.append(i)
            
    theDict = {
        'ATL': [1367],
        'BKN': [1446],
        'BOS': [1497],
        'CHA': [1558],
        'CHI': [1327],
        'CLE': [1434],
        'DAL': [1541],
        'DEN': [1439],
        'DET': [1517],
        'GSW': [1347],
        'HOU': [1493],
        'IND': [1606],
        'LAC': [1277],
        'LAL': [1658],
        'MEM': [1371],
        'MIA': [1575],
        'MIL': [1523],
        'MIN': [1551],
        'NOP': [1500],
        'NYK': [1555],
        'OKC': [1510],
        'ORL': [1518],
        'PHI': [1563],
        'PHX': [1593],
        'POR': [1644],
        'SAC': [1536],
        'SAS': [1595],
        'TOR': [1487],
        'UTA': [1560],
        'WAS': [1436],
    }
    
    for i in listOfSeasons:
        seasonDf = df.loc[df['Season']==i,:]#.reset_index(drop=True,inplace=True)
        
        if i != '00-01':
            for j in listOfTeams:
                teamEloList = theDict.get(j)
                teamElo = teamEloList[-1]
                newTeamElo = int((teamElo*0.75)+(1505*0.25))
                teamEloList.append(newTeamElo)
                theDict.update({j:teamEloList})
                
        for index, r in enumerate(seasonDf['Date']):
            seasonDf.iloc[index,6] = theDict.get(seasonDf.iloc[index,4])[-1]
            seasonDf.iloc[index,129] = theDict.get(seasonDf.iloc[index,127])[-1]
            row = list(seasonDf.iloc[index,:])
            newHome, newAway = newElo(row,theDict)
            homeEloList = theDict.get(newHome[0])
            awayEloList = theDict.get(newAway[0])
            homeEloList.append(newHome[1])
            awayEloList.append(newAway[1])
            theDict.update({newHome[0]:homeEloList})
            theDict.update({newAway[0]:awayEloList})

        
        df.loc[df['Season']==i,:] = seasonDf
        
    return df

In [73]:
newData = addElos(data)
newData

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Date,GameID,Season,HomeAway,HomeTeam,HomeWin,Elo,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXHomeAway,XXAwayTeam,AwayWin,XXElo,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBallTurnovers,XXLiveBallTurnoverPc

In [74]:
elo1 = list(newData['Elo'])
elo2 = list(newData['XXElo'])
avgs = pd.read_csv(r'C:\Users\marte\OneDrive\Documents\Projects!\NBA Model\averagesFinal.csv')

In [75]:
avgs.insert(loc=6,column='Elo',value=elo1)
avgs.insert(loc=129,column='XXElo',value=elo2)
avgs

,Date,GameID,Season,HomeAway,HomeTeam,Wins,Elo,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXHomeAway,XXAwayTeam,XXWins,XXElo,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBallTurnovers,XXLiveBallTurnoverPct,XX

In [76]:
avgs.rename(columns={'HomeTeam':'Team','XXAwayTeam':'XXTeam','HomeAway':'H/A','XXHomeAway':'XXH/A'},inplace=True)
avgs

,Date,GameID,Season,H/A,Team,Wins,Elo,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXH/A,XXTeam,XXWins,XXElo,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBallTurnovers,XXLiveBallTurnoverPct,XXLostBallTurnovers,

In [77]:
avgs.to_csv(r'C:\Users\marte\OneDrive\Documents\Projects!\NBA Model\averagesFinal.csv', index=False)
newData.to_csv(r'C:\Users\marte\OneDrive\Documents\Projects!\NBA Model\CleanData.csv', index=False)